In [1]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import psycopg2
import yaml
import matplotlib.pyplot as plt
import numpy as np

In [2]:
with open('config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
db_string = f"postgresql://{config['user_name']}:{config['db_password']}@{config['host_info']}/{config['db_name']}"
# Create the engine
engine = create_engine(db_string)
# Connect to PostgreSQL server

dbConnection = engine.connect();

In [3]:
import pandas as pd
pgadmin_read_df = pd.read_sql("select * from rent_data_tab", dbConnection);

In [4]:
xx_df = pgadmin_read_df[pgadmin_read_df.zipcode == 77084 ].set_index('Date')
xx_df

,index,zipcode,countyname,city,state,rent,latitude,Long
Date,,,,,,,,
2015-03-31,93,77084,Harris County,Houston,TX,1192.210628,29.844,-95.6623
2015-04-30,94,77084,Harris County,Houston,TX,1204.856444,29.844,-95.6623
2015-05-31,95,77084,Harris County,Houston,TX,1206.741732,29.844,-95.6623
2015-06-30,96,77084,Harris County,Houston,TX,1214.043599,29.844,-95.6623
2015-07-31,97,77084,Harris County,Houston,TX,1216.407630,29.844,-95.6623
...,...,...,...,...,...,...,...,...
2022-07-31,181,77084,Harris County,Houston,TX,1684.885062,29.844,-95.6623
2022-08-31,182,77084,Harris County,Houston,TX,1667.298960,29.844,-95.6623
2022-09-30,183,77084,Harris County,Houston,TX,1650.829787,29.844,-95.6623


In [5]:
pg_df= pgadmin_read_df[["zipcode","Date","rent"]].copy()

In [6]:
zip_list=pgadmin_read_df["zipcode"].unique().tolist()
# zip_list.index(77084)
# zip_list = [77084]

In [7]:
# for j in zip_list[:5]:
#     print ( j )

In [8]:
pg_df.describe()

,zipcode,rent
count,84444.000000,84444.000000
mean,52335.020925,1747.798967
std,30992.458423,735.547560
min,2111.000000,448.487272
25%,28402.500000,1267.575077
50%,37041.000000,1579.556267
75%,85205.250000,2075.269329
max,98661.000000,14894.987860


In [9]:
pg_df.head()

,zipcode,Date,rent
0,11385,2015-03-31,2182.238737
1,11385,2015-04-30,2222.635952
2,11385,2015-05-31,2249.036093
3,11385,2015-06-30,2271.809425
4,11385,2015-07-31,2281.285678


In [15]:
from pmdarima import auto_arima
# Ignore harmless warnings 
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from math import sqrt
import pickle


Best_model = {}
seasonal_order = {}
train_data = {}
test_data = {}
models = {}
for j in zip_list[:100]:
    print ( j )
    pg_df = pgadmin_read_df[pgadmin_read_df.zipcode == j ].set_index('Date')
    data = pg_df["rent"]
    train=data.iloc[:-20]
    test=data.iloc[-20:]
    stepwise_fit = auto_arima(pg_df["rent"], trace=False,supress_warnings=True)
    Best_model[j]=stepwise_fit.get_params()['order']#     print(pg_df.count())
    seasonal_order[j]=stepwise_fit.get_params()['seasonal_order']#     print(pg_df.count())
    model=SARIMAX(train,order = Best_model[j], seasonal_order=(1,1,0,12))
    model=model.fit()
    models[j] = model
    train_data[j] = train
    test_data[j] = test
    future_pred=models[j].predict(start=93,end=93+30,type="levels")
    plt.figure(figsize=(10,3))
    plt.plot(train_data[j].index, train_data[j].values,label="Train Data" )
    plt.plot(test_data[j].index, test_data[j].values,label="Test Data"  )
    plt.plot(future_pred.index, future_pred.values,label="Predict Data"  )
    plt.legend( loc="upper left")
    start=len (train_data[j])
    end=len(train_data[j])+len(test_data[j])-1
    pred=models[j].predict(start=start,end=end,type="levels")
    rmse=sqrt(mean_squared_error(pred,test_data[j]))
    print( f' For Zipcode {j} RMSE Score is {int(rmse)}' )
    if int(rmse) < 250:
        # Save the image of the model
        plt.savefig(f'Images/{j}_{int(rmse)}.jpg')
        
        # Save the model in the Model folder
        pickle.dump(models[j], open(f'Models/{j}.pkl', 'wb'))
        
        # Save the predicted values in the Database table.
        stodf = future_pred.to_frame().reset_index()
        stodf.rename(columns = {'predicted_mean':'FutureRent' , 'index':'month'},inplace=True)
        stodf = stodf.assign(zipcode=j)
        columnsTitles = ['zipcode', 'month', 'FutureRent']
        stodf = stodf.reindex(columns=columnsTitles)
        try:
            stodf.to_sql(name="predicted_rent_data_tab", con=engine,if_exists='append',index=False)
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
#             engine.rollback()
            
            
    plt.close()    
        

11385
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.62937D+00    |proj g|=  1.05298D-01

At iterate    5    f=  3.61352D+00    |proj g|=  1.48245D-02

At iterate   10    f=  3.59403D+00    |proj g|=  1.01407D-02

At iterate   15    f=  3.59375D+00    |proj g|=  1.43203D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6     15     17      1     0     0   1.432D-05   3.594D+00
  F =   3.5937473082161144     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


 This problem is unconstrained.


 For Zipcode 11385 RMSE Score is 518
77084
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.05740D+00    |proj g|=  8.52031D-02

At iterate    5    f=  3.04267D+00    |proj g|=  4.19192D-02

At iterate   10    f=  3.04069D+00    |proj g|=  5.27773D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     10     12      1     0     0   5.278D-06   3.041D+00
  F =   3.0406862570222657     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
 For Zipcode 77084 RMSE Sco

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.64018D+00    |proj g|=  1.46710D-01

At iterate    5    f=  3.62713D+00    |proj g|=  9.70851D-03

At iterate   10    f=  3.62591D+00    |proj g|=  1.69886D-03

At iterate   15    f=  3.62556D+00    |proj g|=  2.98476D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    7     18     22      1     0     0   3.419D-05   3.626D+00
  F =   3.6255513490205793     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


...

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.16608D+00    |proj g|=  4.16131D-01

At iterate    5    f=  3.12688D+00    |proj g|=  6.12702D-02

At iterate   10    f=  3.08591D+00    |proj g|=  1.18554D-01

At iterate   15    f=  3.07947D+00    |proj g|=  4.59512D-02

At iterate   20    f=  3.06020D+00    |proj g|=  2.08965D-02

At iterate   25    f=  3.05456D+00    |proj g|=  2.75748D-02

At iterate   30    f=  3.05360D+00    |proj g|=  7.35957D-03

At iterate   35    f=  3.05033D+00    |proj g|=  5.04482D-02

At iterate   40    f=  3.05026D+00    |proj g|=  9.19036D-04

At iterate   45    f=  3.05026D+00    |proj g|=  9.22710D-04

At iterate   50    f=  3.05026D+00    |proj g|=  1.08278D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cau

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.31563D+00    |proj g|=  5.74319D-02

At iterate    5    f=  3.30610D+00    |proj g|=  2.84484D-02

At iterate   10    f=  3.30161D+00    |proj g|=  1.02349D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     11     14      1     0     0   2.421D-07   3.302D+00
  F =   3.3016068644523280     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
 For Zipcode 22191 RMSE Score is 172
85308
RUNNING THE L-BFGS-B CODE



 This problem is unconstrained.


In [12]:
future_pred

93     1568.568405
94     1610.356348
95     1632.378687
96     1659.605534
97     1630.363692
98     1623.619665
99     1627.001485
100    1656.992503
101    1661.330187
102    1637.426438
103    1591.069013
104    1554.043978
105    1506.357177
106    1547.694838
107    1568.994174
108    1597.993582
109    1568.797396
110    1562.315309
111    1564.280985
112    1594.523803
113    1598.825599
114    1575.967909
115    1529.182517
116    1492.119941
117    1443.712921
118    1485.548615
119    1506.874050
120    1536.337332
121    1506.650917
122    1500.189774
123    1501.874469
Name: predicted_mean, dtype: float64

In [ ]:
# train_data
# print(test_data[77084])
# type(test_data[77084])

In [ ]:
# models[j]

In [ ]:
# start=len (train_data[j])
# end=len(train_data[j])+len(test_data[j])-1
# pred=models[j].predict(start=start,end=end,type="levels")
# print(pred)
# # pred.index=pred.index[start:end+1] # To set the date value as an index
# rmse=sqrt(mean_squared_error(pred,test_data[j]))
# print(rmse)

In [ ]:

# tick_locations = [value for value in test_data[j].index ]

# plt.figure(figsize=(10,3))
# plt.xticks(test_data[j].index,test_data[j].index, rotation='vertical')
# plt.title("Rent Increase Prediction")
# plt.xlabel("Month ")
# plt.ylabel("Rent")
# plt.plot(test_data[j].index, test_data[j].values )
# plt.show()

In [ ]:

# # tick_locations = [value for value in train_data[j].index ]

# plt.figure(figsize=(10,3))
# # plt.xticks(train_data[j].index,train_data[j].index, rotation='vertical')
# # plt.title("Rent Increase Prediction")
# # plt.xlabel("Month ")
# # plt.ylabel("Rent")
# plt.plot(train_data[j].index, train_data[j].values )
# plt.plot(test_data[j].index, test_data[j].values )
# plt.show()

In [ ]:
# pg_df["zipcode"].unique().tolist()

In [ ]:
# future_pred=models[j].predict(start=93,end=93+30,type="levels")

In [ ]:
# future_pred

In [ ]:
# plt.figure(figsize=(10,3))
# # plt.xticks(train_data[j].index,train_data[j].index, rotation='vertical')
# # plt.title("Rent Increase Prediction")
# # plt.xlabel("Month ")
# # plt.ylabel("Rent")
# plt.plot(train_data[j].index, train_data[j].values,label="Train Data" )
# plt.plot(test_data[j].index, test_data[j].values,label="Test Data"  )
# plt.plot(future_pred.index, future_pred.values,label="Predict Data"  )
# plt.legend( loc="upper left")
# # plt.show()
# plt.savefig(f'{j}.jpg')
# plt.close()

In [ ]:
# type(future_pred)

In [ ]:
# pd.DataFrame([future_pred])

In [ ]:
# stodf = future_pred.to_frame().reset_index()
# stodf.rename(columns = {'predicted_mean':'FutureRent' , 'index':'month'},inplace=True)
# stodf = stodf.assign(zipcode=j)
# columnsTitles = ['zipcode', 'month', 'FutureRent']
# stodf = stodf.reindex(columns=columnsTitles)
# stodf.to_sql(name="predicted_rent_data_tab", con=engine,if_exists='replace',index=False)

In [ ]:
#  stodf.to_sql(name="predicted_rent_data_tab", con=engine,if_exists='replace',index=False)

In [ ]:
# models[j]